# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 11:58AM,239763,16,SAR00000343_CD,Sartorius Stedim North America,Released
1,Jun 16 2022 11:57AM,239762,16,SAR00000343,Sartorius Stedim North America,Released
2,Jun 16 2022 11:57AM,239761,15,8108570-BO,"Laser Pharmaceuticals, LLC",Released
3,Jun 16 2022 11:57AM,239761,15,8108609-BO,"Laser Pharmaceuticals, LLC",Released
4,Jun 16 2022 11:57AM,239761,15,8109225-BO,"Laser Pharmaceuticals, LLC",Released
5,Jun 16 2022 11:57AM,239761,15,8108571-BO,"Laser Pharmaceuticals, LLC",Released
6,Jun 16 2022 11:57AM,239761,15,8108575-BO,"Laser Pharmaceuticals, LLC",Released
7,Jun 16 2022 11:57AM,239761,15,8139012-BO,"Laser Pharmaceuticals, LLC",Released
8,Jun 16 2022 11:57AM,239761,15,8139013-BO,"Laser Pharmaceuticals, LLC",Released
9,Jun 16 2022 11:57AM,239761,15,8139014-BO,"Laser Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,239759,Released,2
46,239760,Released,2
47,239761,Released,12
48,239762,Released,1
49,239763,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239759,NaN,NaN,2.0
239760,NaN,NaN,2.0
239761,NaN,NaN,12.0
239762,NaN,NaN,1.0
239763,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,21.0,52.0,1.0
239659,0.0,0.0,1.0
239660,0.0,0.0,1.0
239665,0.0,1.0,0.0
239672,0.0,0.0,29.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,21,52,1
239659,0,0,1
239660,0,0,1
239665,0,1,0
239672,0,0,29


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,21,52,1
1,239659,0,0,1
2,239660,0,0,1
3,239665,0,1,0
4,239672,0,0,29


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,21,52,1
1,239659,,,1
2,239660,,,1
3,239665,,1,
4,239672,,,29


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America
1,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America
2,Jun 16 2022 11:57AM,239761,15,"Laser Pharmaceuticals, LLC"
14,Jun 16 2022 11:56AM,239760,15,"Carwin Pharmaceutical Associates, LLC"
16,Jun 16 2022 11:55AM,239759,15,"Brookfield Pharmaceuticals, LLC"
18,Jun 16 2022 11:53AM,239758,15,"Alliance Pharma, Inc."
26,Jun 16 2022 11:45AM,239757,10,Ivaoes Animal Health
28,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America
29,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America
30,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,,,1
1,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,,,1
2,Jun 16 2022 11:57AM,239761,15,"Laser Pharmaceuticals, LLC",,,12
3,Jun 16 2022 11:56AM,239760,15,"Carwin Pharmaceutical Associates, LLC",,,2
4,Jun 16 2022 11:55AM,239759,15,"Brookfield Pharmaceuticals, LLC",,,2
5,Jun 16 2022 11:53AM,239758,15,"Alliance Pharma, Inc.",,,8
6,Jun 16 2022 11:45AM,239757,10,Ivaoes Animal Health,,,2
7,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,,,1
8,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,,,1
9,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,1,,
1,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,1,,
2,Jun 16 2022 11:57AM,239761,15,"Laser Pharmaceuticals, LLC",12,,
3,Jun 16 2022 11:56AM,239760,15,"Carwin Pharmaceutical Associates, LLC",2,,
4,Jun 16 2022 11:55AM,239759,15,"Brookfield Pharmaceuticals, LLC",2,,
5,Jun 16 2022 11:53AM,239758,15,"Alliance Pharma, Inc.",8,,
6,Jun 16 2022 11:45AM,239757,10,Ivaoes Animal Health,2,,
7,Jun 16 2022 11:38AM,239755,16,Sartorius Stedim North America,1,,
8,Jun 16 2022 11:36AM,239754,16,Sartorius Stedim North America,1,,
9,Jun 16 2022 11:35AM,239753,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,1,,
1,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,1,,
2,Jun 16 2022 11:57AM,239761,15,"Laser Pharmaceuticals, LLC",12,,
3,Jun 16 2022 11:56AM,239760,15,"Carwin Pharmaceutical Associates, LLC",2,,
4,Jun 16 2022 11:55AM,239759,15,"Brookfield Pharmaceuticals, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,1.0,NaN,NaN
2,Jun 16 2022 11:57AM,239761,15,"Laser Pharmaceuticals, LLC",12.0,NaN,NaN
3,Jun 16 2022 11:56AM,239760,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN,NaN
4,Jun 16 2022 11:55AM,239759,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,1.0,0.0,0.0
2,Jun 16 2022 11:57AM,239761,15,"Laser Pharmaceuticals, LLC",12.0,0.0,0.0
3,Jun 16 2022 11:56AM,239760,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0,0.0
4,Jun 16 2022 11:55AM,239759,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2157486,80.0,6.0,7.0
12,719245,3.0,1.0,0.0
15,959038,24.0,0.0,0.0
16,1198733,5.0,0.0,0.0
18,239686,1.0,0.0,0.0
19,958825,11.0,3.0,30.0
20,2157270,52.0,69.0,21.0
22,958768,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2157486,80.0,6.0,7.0
1,12,719245,3.0,1.0,0.0
2,15,959038,24.0,0.0,0.0
3,16,1198733,5.0,0.0,0.0
4,18,239686,1.0,0.0,0.0
5,19,958825,11.0,3.0,30.0
6,20,2157270,52.0,69.0,21.0
7,22,958768,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,80.0,6.0,7.0
1,12,3.0,1.0,0.0
2,15,24.0,0.0,0.0
3,16,5.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,11.0,3.0,30.0
6,20,52.0,69.0,21.0
7,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,80.0
1,12,Released,3.0
2,15,Released,24.0
3,16,Released,5.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,30.0,21.0,0.0
Executing,6.0,1.0,0.0,0.0,0.0,3.0,69.0,0.0
Released,80.0,3.0,24.0,5.0,1.0,11.0,52.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,7.0,0.0,0.0,0.0,0.0,30.0,21.0,0.0
1,Executing,6.0,1.0,0.0,0.0,0.0,3.0,69.0,0.0
2,Released,80.0,3.0,24.0,5.0,1.0,11.0,52.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,7.0,0.0,0.0,0.0,0.0,30.0,21.0,0.0
1,Executing,6.0,1.0,0.0,0.0,0.0,3.0,69.0,0.0
2,Released,80.0,3.0,24.0,5.0,1.0,11.0,52.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()